In [1]:
from numpy import save
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def datos(file):
    data = mne.io.read_raw_edf(f'{file}.edf')
    raw_data = data.get_data()
    events, events_id = mne.events_from_annotations(data)
    tmax = 30. - 1. / data.info['sfreq']  # tmax in included

    epochs_train = mne.Epochs(raw=data, events=events,
                              event_id=events_id, tmin=0., tmax=tmax, baseline=None, event_repeated='merge')
    patatuela = epochs_train.load_data()
    id_to_class_dict = {value:key for key, value in patatuela.event_id.items()}

    objective_stages = ['Sleep stage N1','Sleep stage N2','Sleep stage N3','Sleep stage R','Sleep stage W']
    epochs_to_df = [a for a in patatuela]
    epochs_to_df = []
    label_to_df = []
    for epoch, label in zip(patatuela, patatuela.events[:,-1]):
        # print(label)
        label = id_to_class_dict[label]
        if label in objective_stages:
            # print("True")
            epochs_to_df.append(epoch)
            label_to_df.append(label)
    dict_to_df = {"Epoch":epochs_to_df,"Label":label_to_df}
    df1=pd.DataFrame(dict_to_df)
    valores=df1.Epoch.values
    channels = data.ch_names
    diccionarios=[dict(zip(channels, sublist)) for sublist in valores]
    c3=[d['C3'] for d in diccionarios]
    c4=[d['C4'] for d in diccionarios]
    o1=[d['O1'] for d in diccionarios]
    o2=[d['O2'] for d in diccionarios]
    loc=[d['LOC'] for d in diccionarios]
    roc=[d['ROC'] for d in diccionarios]
    chin=[d['CHIN1'] for d in diccionarios]
    matriz=np.array([c3,c4,o1,o2,loc,roc,chin])

    name=file
    save(f'datos_{name}.npy', matriz)
    df1.Label.to_csv(f'Etiquetas_{name}.csv',index=None)

    return matriz, df1.Label

In [2]:
filess=[f'PSG{i}' for i in range(1,15)]

In [55]:
# for fil in filess:
#     print(datos(fil))    #ejecutar esto para que te cree los archivos npy y csv de etiquetas pero lo dejo así para no verlo por pantalla

In [12]:
datos=[f'datos_PSG{i}.npy' for i in range(1,15)]

In [16]:
matriz_tocha=[np.load(d) for d in datos]

In [24]:
np.shape(matriz_tocha[0])

(7, 882, 15360)

In [56]:
# matriz_tocha[0:10]

In [43]:
etiquetas=[f'Etiquetas_PSG{i}.csv' for i in range(1,15)]

In [53]:
et_todas=[pd.read_csv(d).iloc[:,0] for d in etiquetas]

In [57]:
# et_todas